In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, random, time

import warnings
warnings.filterwarnings('ignore')

In [2]:
starting_dir = os.getcwd()
print ("starting_dir:" + str(starting_dir))

# mac - phase I & II - morpho
complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/Phase_I_II/input/morpho/tall/zf_morphology_data_335_chemicals_2020DEC16.csv'

# mac - phase III - morpho - 23 endpoints - full
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/full/tall/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_full_w_23_endpoints.csv'

# mac - phase III - morpho - 23 endpoints - devel
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/devel/tall/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_devel_w_23_endpoints.csv'

# mac - phase III - morpho - 14 endpoints - full
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/full/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_full_w_14_endpoints.csv'

# mac - phase III - morpho - 14 endpoints - devel
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/devel/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_devel_w_14_endpoints.csv'

morph_data = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)

morph_data.head()

starting_dir:/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/1_reformat/all_targets/morpho/old_archive


,chemical.id,bottle.id,conc,plate.id,well,date,endpoint,value
0,53,C96423,0.0,9289,A06,20151105,MO24,1.0
1,53,C96423,0.0,9289,B06,20151105,MO24,0.0
2,53,C96423,0.0,9289,C06,20151105,MO24,0.0
3,53,C96423,0.0,9289,D06,20151105,MO24,0.0
4,53,C96423,0.0,9289,E06,20151105,MO24,0.0


In [70]:
# just TX part
#TX_bottle_id_morph_data = morph_data[morph_data['bottle.id'].str.contains("^TX")]
#TX_bottle_id_morph_data = morph_data[morph_data['bottle.id'].str.contains("TX$")]
#TX_bottle_id_morph_data.head()

In [71]:
# basic analysis
#new_morph_data = morph_data.loc[morph_data['chemical.id'] == 197,['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
#new_morph_data.head()
#morph_data_w_end = new_morph_data.loc[new_morph_data['endpoint'] == 'MO24',['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
#morph_data_w_end.head()
#np.sum(morph_data['value'] == 1)
#np.sum(morph_data['value'] == 0) # -> 82%
#np.sum(np.isnan(morph_data['value']))
#morph_data.shape

In [3]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'bottle.id', 'plate.id', 'well', 'endpoint', 'value'] 
# 'bottle.id' is added since Lisa wants to use it to join with behavioral data

morph_data_select = morph_data.loc[:,columns_to_keep]
morph_data_select.head()

,chemical.id,conc,bottle.id,plate.id,well,endpoint,value
0,53,0.0,C96423,9289,A06,MO24,1.0
1,53,0.0,C96423,9289,B06,MO24,0.0
2,53,0.0,C96423,9289,C06,MO24,0.0
3,53,0.0,C96423,9289,D06,MO24,0.0
4,53,0.0,C96423,9289,E06,MO24,0.0


In [4]:
print ("<before dropping duplicates> len(morph_data_select):" + str(len(morph_data_select)))
morph_data_select = morph_data_select.drop_duplicates()
print ("<after dropping duplicates>  len(morph_data_select):" + str(len(morph_data_select)))

<before dropping duplicates> len(morph_data_select):1600800
<after dropping duplicates>  len(morph_data_select):1600800


In [5]:
#skip
#'''
unique_endpoints = np.unique(morph_data_select['endpoint'])
print(unique_endpoints)
print(len(unique_endpoints))
#'''

['AXIS' 'BRAI' 'CFIN' 'CIRC' 'DNC_' 'DP24' 'EYE_' 'JAW_' 'MO24' 'MORT'
 'NC24' 'NC__' 'OTIC' 'PE__' 'PFIN' 'PIG_' 'SM24' 'SNOU' 'SOMI' 'SWIM'
 'TRUN' 'TR__' 'YSE_']
23


In [8]:
morph_220 = morph_data_select.loc[morph_data['chemical.id'] == 220,:]
morph_220.to_csv('220.csv', index=False)


In [7]:
# tall -> wide
start_time = time.time()

reformatted = pd.DataFrame()

full_devel = "full"
#full_devel = "devel"

chemical_id_from_here = np.unique(morph_data_select['chemical.id'])

if (full_devel == "devel"):
    randomly_chosen = random.sample(set(chemical_id_from_here), 1)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])
        
for chemical_index in chemical_id_from_here:
    #if (chemical_index != 220):
    if (chemical_index != 53):
        continue
    print("\nchemical_index:" + str(chemical_index))        
    
    morph_data_chemical = morph_data_select.loc[morph_data['chemical.id'] == chemical_index,:]
    display(len(morph_data_chemical))
    display(morph_data_chemical.head())
    
    # Append chemical_plate_well as a unique identifier
    morph_data_chemical.insert(0, 'chemical_plate_well', morph_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    #print ("len(morph_data_chemical.chemical_plate_well):" + str(len(morph_data_chemical.chemical_plate_well)))    
    for cpw in np.unique(morph_data_chemical.chemical_plate_well):

        temp_df = morph_data_chemical.loc[morph_data_chemical.chemical_plate_well == cpw,:]
        #display(len(temp_df))
        #display(temp_df)
        
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])

        for name, group in temp_df_grouped:
            #print ("\nlen(group.endpoint):" + str(len(group.endpoint)))
            #print (group.endpoint)
            #print (group)
            #write_this = str(chemical_index) + "," + str(len(group.endpoint)) + "\n"
            #endpoint_count.write(write_this)
            #if(len(group.endpoint) == 23):
            #    continue
            #else:
            #    print ("len(group.endpoint):" + str(len(group.endpoint)))
            
            #'''
            if(len(group.endpoint) == 14):
                temp = pd.DataFrame(
                    {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRN_': temp_df.value[temp_df.endpoint == 'BRN_'].values,
                        'CRAN': temp_df.value[temp_df.endpoint == 'CRAN'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EDEM': temp_df.value[temp_df.endpoint == 'EDEM'].values,
                        'LTRK': temp_df.value[temp_df.endpoint == 'LTRK'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'MUSC': temp_df.value[temp_df.endpoint == 'MUSC'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'SKIN': temp_df.value[temp_df.endpoint == 'SKIN'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'TCHR': temp_df.value[temp_df.endpoint == 'TCHR'].values,
                    })
                reformatted = pd.concat([reformatted, temp])
            elif(len(group.endpoint) == 23):
                temp = pd.DataFrame(
                    {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRAI': temp_df.value[temp_df.endpoint == 'BRAI'].values,
                        'CFIN': temp_df.value[temp_df.endpoint == 'CFIN'].values,
                        'CIRC': temp_df.value[temp_df.endpoint == 'CIRC'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EYE_': temp_df.value[temp_df.endpoint == 'EYE_'].values,
                        'JAW_': temp_df.value[temp_df.endpoint == 'JAW_'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'NC24': temp_df.value[temp_df.endpoint == 'NC24'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'OTIC': temp_df.value[temp_df.endpoint == 'OTIC'].values,
                        'PE__': temp_df.value[temp_df.endpoint == 'PE__'].values,
                        'PFIN': temp_df.value[temp_df.endpoint == 'PFIN'].values,
                        'PIG_': temp_df.value[temp_df.endpoint == 'PIG_'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'SNOU': temp_df.value[temp_df.endpoint == 'SNOU'].values,
                        'SOMI': temp_df.value[temp_df.endpoint == 'SOMI'].values,
                        'SWIM': temp_df.value[temp_df.endpoint == 'SWIM'].values,
                        'TRUN': temp_df.value[temp_df.endpoint == 'TRUN'].values,
                        'TR__': temp_df.value[temp_df.endpoint == 'TR__'].values,
                        'YSE_': temp_df.value[temp_df.endpoint == 'YSE_'].values,
                    })
                reformatted = pd.concat([reformatted, temp])
            elif(len(group.endpoint) == 30):
                temp = pd.DataFrame(
                    {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRAI': temp_df.value[temp_df.endpoint == 'BRAI'].values,
                        'BRN_': temp_df.value[temp_df.endpoint == 'BRN_'].values,
                        'CFIN': temp_df.value[temp_df.endpoint == 'CFIN'].values,
                        'CIRC': temp_df.value[temp_df.endpoint == 'CIRC'].values,
                        'CRAN': temp_df.value[temp_df.endpoint == 'CRAN'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EDEM': temp_df.value[temp_df.endpoint == 'EDEM'].values,
                        'EYE_': temp_df.value[temp_df.endpoint == 'EYE_'].values,
                        'JAW_': temp_df.value[temp_df.endpoint == 'JAW_'].values,
                        'LTRK': temp_df.value[temp_df.endpoint == 'LTRK'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'MUSC': temp_df.value[temp_df.endpoint == 'MUSC'].values,
                        'NC24': temp_df.value[temp_df.endpoint == 'NC24'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'OTIC': temp_df.value[temp_df.endpoint == 'OTIC'].values,
                        'PE__': temp_df.value[temp_df.endpoint == 'PE__'].values,
                        'PFIN': temp_df.value[temp_df.endpoint == 'PFIN'].values,
                        'PIG_': temp_df.value[temp_df.endpoint == 'PIG_'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'SNOU': temp_df.value[temp_df.endpoint == 'SNOU'].values,
                        'SOMI': temp_df.value[temp_df.endpoint == 'SOMI'].values,
                        'SWIM': temp_df.value[temp_df.endpoint == 'SWIM'].values,
                        'TCHR': temp_df.value[temp_df.endpoint == 'TCHR'].values,
                        'TRUN': temp_df.value[temp_df.endpoint == 'TRUN'].values,
                        'TR__': temp_df.value[temp_df.endpoint == 'TR__'].values,
                        'YSE_': temp_df.value[temp_df.endpoint == 'YSE_'].values,
                    })
                reformatted = pd.concat([reformatted, temp])
            else:
                print ("len(group.endpoint):" + str(len(group.endpoint)))
               # a=b
            #    '''                
#endpoint_count.close()
                  
    
################## possible manual update of entries
find_this = reformatted.loc[(reformatted['chemical.id'] == 746) \
                & (reformatted['plate.id'] == 12953) \
                & (reformatted['well'] == "E03"), 'MORT'] 
if (len(find_this) != 0):
    print ("for phase I, II, automatically fix small manual mistakes")
    reformatted.loc[(reformatted['chemical.id'] == 746) \
                & (reformatted['plate.id'] == 12953) \
                & (reformatted['well'] == "E03"), 'MORT'] = 0

    
find_this = reformatted.loc[(reformatted['chemical.id'] == 1171) \
                    & (reformatted['plate.id'] == 15205) \
                    & (reformatted['well'] == "D02"), 'MORT']
if (len(find_this) != 0):
    print ("for phase I, II, automatically fix small manual mistakes")
    reformatted.loc[(reformatted['chemical.id'] == 1171) \
                    & (reformatted['plate.id'] == 15205) \
                    & (reformatted['well'] == "D02"), 'MORT'] = 0

       
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("tall-> wide done, it took:"+str(time_took)) # took 13 minutes for 344 chemicals


chemical_index:53


4416

,chemical.id,conc,bottle.id,plate.id,well,endpoint,value
0,53,0.0,C96423,9289,A06,MO24,1.0
1,53,0.0,C96423,9289,B06,MO24,0.0
2,53,0.0,C96423,9289,C06,MO24,0.0
3,53,0.0,C96423,9289,D06,MO24,0.0
4,53,0.0,C96423,9289,E06,MO24,0.0


tall-> wide done, it took:1.9 seconds


In [77]:
'''endpoint_investigate_data = pd.read_csv(endpoint_report_filename, header = 0)

print ("<before dropping duplicates> len(endpoint_investigate_data):" + str(len(endpoint_investigate_data)))
unique_endpoint_investigate_data = endpoint_investigate_data.drop_duplicates()
print ("<after dropping duplicates>  len(unique_endpoint_investigate_data):" + str(len(unique_endpoint_investigate_data)))

#display(unique_endpoint_investigate_data)

#unique_endpoint_investigate_data.to_csv('unique_endpoint_investigate_data.csv', index=False)
'''

'endpoint_investigate_data = pd.read_csv(endpoint_report_filename, header = 0)\n\nprint ("<before dropping duplicates> len(endpoint_investigate_data):" + str(len(endpoint_investigate_data)))\nunique_endpoint_investigate_data = endpoint_investigate_data.drop_duplicates()\nprint ("<after dropping duplicates>  len(unique_endpoint_investigate_data):" + str(len(unique_endpoint_investigate_data)))\n\n#display(unique_endpoint_investigate_data)\n\n#unique_endpoint_investigate_data.to_csv(\'unique_endpoint_investigate_data.csv\', index=False)\n'

In [78]:
#print (reformatted.head())
#print ("reformatted.shape:" + str(reformatted.shape))

In [8]:
reformatted_DNC_0 = pd.DataFrame()
reformatted_DNC_0 = reformatted.loc[reformatted['DNC_'] == 0.0]
print ("reformatted_DNC_0.shape:" + str(reformatted_DNC_0.shape))

reformatted_DNC_0.shape:(192, 28)


In [85]:
reformatted_DNC_empty = pd.DataFrame()
reformatted_DNC_empty = reformatted.loc[reformatted['DNC_'] == '']
print ("reformatted_DNC_empty.shape:" + str(reformatted_DNC_empty.shape))

reformatted_DNC_empty.shape:(0, 28)


In [81]:
#'''
nan_count = 0
zero_count = 0
one_count = 0
total_count = 0
for (columnName, columnData) in reformatted.iteritems(): 
    if (columnName == "chemical.id") or (columnName == "plate.id") or (columnName == "well") or (columnName == "chemical_plate_well") or (columnName == "conc"):
        continue
#    print('Colunm Name : ', columnName) 
#    print('Column Contents : ', columnData.values[i])
#    print (len(columnData.values))
    for i in range(len(columnData.values)):
        total_count += 1
        if (str(columnData.values[i]) == "nan"):
            nan_count += 1
        elif (str(columnData.values[i]) == "0.0"):
            zero_count += 1
        elif (str(columnData.values[i]) == "1.0"):
            one_count += 1
#        print('Column Contents : ', columnData.values[i])
print ("nan_count:" + str(nan_count))
print ("zero_count:" + str(zero_count))
print ("one_count:" + str(one_count))
print ("total_count:" + str(total_count))
#'''

nan_count:123408
zero_count:730792
one_count:22376
total_count:876576


In [82]:
#pd.set_option('display.max_columns', None)

In [9]:
#reformatted.to_csv('344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_wide_DNC_all.csv',index=False)

output_complete_file_path = complete_file_path[:-4] + "_wide_DNC_0_" \
                            + str(full_devel) + ".csv"
display ('output_complete_file_path:', str(output_complete_file_path))

reformatted_DNC_0.to_csv(output_complete_file_path,index=False)

#reformatted_DNC_empty.to_csv('344_zf_morphology_data_phase_1_and_2_-_2020JUNE25_wide_DNC_empty.csv',index=False)

'output_complete_file_path:'

'/Users/kimd999/research/projects/toxicity/per_each_data/Phase_I_II/input/morpho/tall/zf_morphology_data_335_chemicals_2020DEC16_wide_DNC_0_full.csv'